# Sentiment Analysis on IMDB Reviews using LSTM and Keras
created by Dinesh Pasupuleti
<hr>

### Steps
<ol type="1">
    <li>Load the dataset (50K IMDB Movie Review)</li>
    <li>Clean Dataset</li>
    <li>Encode Sentiments</li>
    <li>Split Dataset</li>
    <li>Tokenize and Pad/Truncate Reviews</li>
    <li>Build Architecture/Model</li>
    <li>Train and Test</li>
</ol>

<hr>
<i>Import all the libraries needed</i>

In [1]:
import pandas as pd    # to load dataset
import numpy as np     # for mathematic equation
from nltk.corpus import stopwords   # to get collection of stopwords
from sklearn.model_selection import train_test_split       # for splitting dataset
from tensorflow.keras.preprocessing.text import Tokenizer  # to encode text to int
from tensorflow.keras.preprocessing.sequence import pad_sequences   # to do padding or truncating
from tensorflow.keras.models import Sequential     # the model
from tensorflow.keras.layers import Embedding, LSTM, Dense # layers of the architecture
from tensorflow.keras.callbacks import ModelCheckpoint   # save model
from tensorflow.keras.models import load_model   # load saved model
import re

<hr>
<i>Preview dataset</i>

In [2]:
data = pd.read_csv('C:/Users/User/OneDrive/Desktop/IMDB_Dataset.csv')

print(data)

                                                  review sentiment
0      One of the other reviewers has mentioned that ...  positive
1      A wonderful little production. <br /><br />The...  positive
2      I thought this was a wonderful way to spend ti...  positive
3      Basically there's a family where a little boy ...  negative
4      Petter Mattei's "Love in the Time of Money" is...  positive
...                                                  ...       ...
49995  I thought this movie did a down right good job...  positive
49996  Bad plot, bad dialogue, bad acting, idiotic di...  negative
49997  I am a Catholic taught in parochial elementary...  negative
49998  I'm going to have to disagree with the previou...  negative
49999  No one expects the Star Trek movies to be high...  negative

[50000 rows x 2 columns]


<hr>
<b>Stop Word</b> is a commonly used words in a sentence, usually a search engine is programmed to ignore this words (i.e. "the", "a", "an", "of", etc.)

<i>Declaring the english stop words</i>

In [4]:
english_stops = set(stopwords.words('english'))
print(english_stops)

{'just', 'an', 'you', 'o', 'mightn', 'while', 'into', 'ma', 'shouldn', 'doesn', 'only', "it's", "you're", "couldn't", 'her', 'mustn', 'll', 'other', 'don', 'with', 'yours', 'and', 'by', 'both', 't', 'should', 'from', 'weren', 'so', 'during', 'we', "aren't", 'myself', 'yourselves', 'doing', 'if', 'theirs', 'now', "needn't", 'will', 'at', "mightn't", 'what', 'most', 'am', "should've", "you'll", 'having', 'was', 'some', 'being', 'too', 'this', 'nor', 'is', 'of', 'through', 'our', 'few', "isn't", 'hers', 'are', 'hasn', 'been', 'same', 'do', 'ours', 'on', 'wouldn', 'were', 'because', "wasn't", 'isn', "wouldn't", 've', "don't", "shan't", 'again', 'haven', 'ourselves', 'until', 'm', "hadn't", "won't", "you've", 'had', 'out', 'he', 'about', 'for', 'him', 'themselves', 'it', 'shan', 'very', 'down', 'yourself', 'these', "hasn't", 'wasn', 'when', 'but', 'won', 'they', 'them', 'who', 'needn', 'me', 'or', 'my', 'not', 'as', 'each', 'your', 'his', 'once', 'where', 'more', 'did', 'd', 'own', "haven't

<hr>

### Load and Clean Dataset

In the original dataset, the reviews are still dirty. There are still html tags, numbers, uppercase, and punctuations. This will not be good for training, so in <b>load_dataset()</b> function, beside loading the dataset using <b>pandas</b>, I also pre-process the reviews by removing html tags, non alphabet (punctuations and numbers), stop words, and lower case all of the reviews.

### Encode Sentiments
In the same function, I also encode the sentiments into integers (0 and 1). Where 0 is for negative sentiments and 1 is for positive sentiments.

In [5]:
def load_dataset():
    df = pd.read_csv('C:/Users/User/OneDrive/Desktop/IMDB_Dataset.csv')
    x_data = df['review']       # Reviews/Input
    y_data = df['sentiment']    # Sentiment/Output

    # PRE-PROCESS REVIEW
    x_data = x_data.replace({'<.*?>': ''}, regex = True)          # remove html tag
    x_data = x_data.replace({'[^A-Za-z]': ' '}, regex = True)     # remove non alphabet
    x_data = x_data.apply(lambda review: [w for w in review.split() if w not in english_stops])  # remove stop words
    x_data = x_data.apply(lambda review: [w.lower() for w in review])   # lower case
    
    # ENCODE SENTIMENT -> 0 & 1
    y_data = y_data.replace('positive', 1)
    y_data = y_data.replace('negative', 0)

    return x_data, y_data

x_data, y_data = load_dataset()

print('Reviews')
print(x_data, '\n')
print('Sentiment')
print(y_data)

Reviews
0        [one, reviewers, mentioned, watching, oz, epis...
1        [a, wonderful, little, production, the, filmin...
2        [i, thought, wonderful, way, spend, time, hot,...
3        [basically, family, little, boy, jake, thinks,...
4        [petter, mattei, love, time, money, visually, ...
                               ...                        
49995    [i, thought, movie, right, good, job, it, crea...
49996    [bad, plot, bad, dialogue, bad, acting, idioti...
49997    [i, catholic, taught, parochial, elementary, s...
49998    [i, going, disagree, previous, comment, side, ...
49999    [no, one, expects, star, trek, movies, high, a...
Name: review, Length: 50000, dtype: object 

Sentiment
0        1
1        1
2        1
3        0
4        1
        ..
49995    1
49996    0
49997    0
49998    0
49999    0
Name: sentiment, Length: 50000, dtype: int64


<hr>

### Split Dataset
In this work, I decided to split the data into 80% of Training and 20% of Testing set using <b>train_test_split</b> method from Scikit-Learn. By using this method, it automatically shuffles the dataset. We need to shuffle the data because in the original dataset, the reviews and sentiments are in order, where they list positive reviews first and then negative reviews. By shuffling the data, it will be distributed equally in the model, so it will be more accurate for predictions.

In [6]:
x_train, x_test, y_train, y_test = train_test_split(x_data, y_data, test_size = 0.2)

print('Train Set')
print(x_train, '\n')
print(x_test, '\n')
print('Test Set')
print(y_train, '\n')
print(y_test)

Train Set
23249    [i, sligthly, biased, i, appear, film, loved, ...
23015    [the, original, exploitation, classic, though,...
39158    [the, influences, science, fiction, thriller, ...
32121    [if, one, people, finds, according, jim, great...
1544     [this, excellent, suspenseful, murder, mystery...
                               ...                        
13500    [i, watched, series, tv, absolutely, loved, i,...
48725    [you, got, see, believe, shot, hollywood, stri...
4357     [brokedown, palace, story, two, best, friends,...
5568     [love, jones, cleverly, portrays, young, afric...
28598    [i, went, see, movie, tonight, trying, keep, o...
Name: review, Length: 40000, dtype: object 

48720    [this, first, introduction, world, bollywood, ...
6757     [ok, i, watch, many, horror, movies, reason, f...
33659    [ok, lets, see, what, funny, first, movie, i, ...
44620    [emma, woodhouse, gwyneth, paltrow, shakespear...
28594    [myself, groovadelic, something, pals, get, en...
 

<hr>
<i>Function for getting the maximum review length, by calculating the mean of all the reviews length (using <b>numpy.mean</b>)</i>

In [7]:
def get_max_length():
    review_length = []
    for review in x_train:
        review_length.append(len(review))

    return int(np.ceil(np.mean(review_length)))

<hr>

### Tokenize and Pad/Truncate Reviews
A Neural Network only accepts numeric data, so we need to encode the reviews. I use <b>tensorflow.keras.preprocessing.text.Tokenizer</b> to encode the reviews into integers, where each unique word is automatically indexed (using <b>fit_on_texts</b> method) based on <b>x_train</b>. <br>
<b>x_train</b> and <b>x_test</b> is converted into integers using <b>texts_to_sequences</b> method.

Each reviews has a different length, so we need to add padding (by adding 0) or truncating the words to the same length (in this case, it is the mean of all reviews length) using <b>tensorflow.keras.preprocessing.sequence.pad_sequences</b>.


<b>post</b>, pad or truncate the words in the back of a sentence<br>
<b>pre</b>, pad or truncate the words in front of a sentence

In [8]:
# ENCODE REVIEW
token = Tokenizer(lower=False)    # no need lower, because already lowered the data in load_data()
token.fit_on_texts(x_train)
x_train = token.texts_to_sequences(x_train)
x_test = token.texts_to_sequences(x_test)

max_length = get_max_length()

x_train = pad_sequences(x_train, maxlen=max_length, padding='post', truncating='post')
x_test = pad_sequences(x_test, maxlen=max_length, padding='post', truncating='post')

total_words = len(token.word_index) + 1   # add 1 because of 0 padding

print('Encoded X Train\n', x_train, '\n')
print('Encoded X Test\n', x_test, '\n')
print('Maximum review length: ', max_length)

Encoded X Train
 [[    1 57044  4831 ...     0     0     0]
 [    2   124  2167 ...     0     0     0]
 [    2  7146   968 ...   378     3     7]
 ...
 [19935  5084    13 ...     0     0     0]
 [   41  1143  4528 ...     0     0     0]
 [    1   320    15 ...     0     0     0]] 

Encoded X Test
 [[    8    23  2751 ...     0     0     0]
 [  491     1    34 ...   127     1  1745]
 [  491  1512    15 ...     0     0     0]
 ...
 [ 1941 20949   435 ...     0     0     0]
 [    1    98   170 ...  1411     8 29531]
 [ 1318 10879   848 ... 72729  3977  3671]] 

Maximum review length:  130


<hr>

### Build Architecture/Model
<b>Embedding Layer</b>: in simple terms, it creates word vectors of each word in the <i>word_index</i> and group words that are related or have similar meaning by analyzing other words around them.

<b>LSTM Layer</b>: to make a decision to keep or throw away data by considering the current input, previous output, and previous memory. There are some important components in LSTM.
<ul>
    <li><b>Forget Gate</b>, decides information is to be kept or thrown away</li>
    <li><b>Input Gate</b>, updates cell state by passing previous output and current input into sigmoid activation function</li>
    <li><b>Cell State</b>, calculate new cell state, it is multiplied by forget vector (drop value if multiplied by a near 0), add it with the output from input gate to update the cell state value.</li>
    <li><b>Ouput Gate</b>, decides the next hidden state and used for predictions</li>
</ul>

<b>Dense Layer</b>: compute the input with the weight matrix and bias (optional), and using an activation function. I use <b>Sigmoid</b> activation function for this work because the output is only 0 or 1.

The optimizer is <b>Adam</b> and the loss function is <b>Binary Crossentropy</b> because again the output is only 0 and 1, which is a binary number.

In [9]:
# ARCHITECTURE
EMBED_DIM = 32
LSTM_OUT = 64

model = Sequential()
model.add(Embedding(total_words, EMBED_DIM, input_length = max_length))
model.add(LSTM(LSTM_OUT))
model.add(Dense(1, activation='sigmoid'))
model.compile(optimizer = 'adam', loss = 'binary_crossentropy', metrics = ['accuracy'])

print(model.summary())

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, 130, 32)           2961152   
_________________________________________________________________
lstm (LSTM)                  (None, 64)                24832     
_________________________________________________________________
dense (Dense)                (None, 1)                 65        
Total params: 2,986,049
Trainable params: 2,986,049
Non-trainable params: 0
_________________________________________________________________
None


<hr>

### Training
For training, it is simple. We only need to fit our <b>x_train</b> (input) and <b>y_train</b> (output/label) data. For this training, I use a mini-batch learning method with a <b>batch_size</b> of <i>128</i> and <i>5</i> <b>epochs</b>.

Also, I added a callback called **checkpoint** to save the model locally for every epoch if its accuracy improved from the previous epoch.

In [15]:
checkpoint = ModelCheckpoint(
    'LSTM.h5',
    monitor='accuracy',
    save_best_only=True,
    verbose=1
)

In [16]:
model.fit(x_train, y_train, batch_size = 128, epochs = 5, callbacks=[checkpoint])

Epoch 1/5
313/313 [==============================] - ETA: 0s - loss: 0.0462 - accuracy: 0.9886
Epoch 00001: accuracy improved from -inf to 0.98862, saving model to LSTM.h5
313/313 [==============================] - 56s 178ms/step - loss: 0.0462 - accuracy: 0.9886
Epoch 2/5
313/313 [==============================] - ETA: 0s - loss: 0.0392 - accuracy: 0.9903
Epoch 00002: accuracy improved from 0.98862 to 0.99032, saving model to LSTM.h5
313/313 [==============================] - 56s 178ms/step - loss: 0.0392 - accuracy: 0.9903
Epoch 3/5
313/313 [==============================] - ETA: 0s - loss: 0.0279 - accuracy: 0.9936
Epoch 00003: accuracy improved from 0.99032 to 0.99357, saving model to LSTM.h5
313/313 [==============================] - 56s 179ms/step - loss: 0.0279 - accuracy: 0.9936
Epoch 4/5
313/313 [==============================] - ETA: 0s - loss: 0.0328 - accuracy: 0.9916
Epoch 00004: accuracy did not improve from 0.99357
313/313 [==============================] - 55s 176ms/ste

<hr>

### Testing
To evaluate the model, we need to predict the sentiment using our <b>x_test</b> data and comparing the predictions with <b>y_test</b> (expected output) data. Then, we calculate the accuracy of the model by dividing numbers of correct prediction with the total data. Resulted an accuracy of <b>85.45%</b>

In [17]:
y_pred = model.predict_classes(x_test, batch_size = 128)

true = 0
for i, y in enumerate(y_test):
    if y == y_pred[i]:
        true += 1

print('Correct Prediction: {}'.format(true))
print('Wrong Prediction: {}'.format(len(y_pred) - true))
print('Accuracy: {}'.format(true/len(y_pred)*100))

Correct Prediction: 8545
Wrong Prediction: 1455
Accuracy: 85.45


---

### Load Saved Model

Load saved model and use it to predict a movie review statement's sentiment (positive or negative).

In [18]:
loaded_model = load_model('LSTM.h5')

Receives a review as an input to be predicted

In [19]:
review = str(input('Movie Review: '))

Movie Review: I have never seen such an amazing film since I saw The Shawshank Redemption. Shawshank encompasses friendships, hardships, hopes, and dreams. And what is so great about the movie is that it moves you, it gives you hope. Even though the circumstances between the characters and the viewers are quite different, you don't feel that far removed from what the characters are going through.


The input must be pre processed before it is passed to the model to be predicted

In [20]:
# Pre-process input
regex = re.compile(r'[^a-zA-Z\s]')
review = regex.sub('', review)
print('Cleaned: ', review)

words = review.split(' ')
filtered = [w for w in words if w not in english_stops]
filtered = ' '.join(filtered)
filtered = [filtered.lower()]

print('Filtered: ', filtered)

Cleaned:  I have never seen such an amazing film since I saw The Shawshank Redemption Shawshank encompasses friendships hardships hopes and dreams And what is so great about the movie is that it moves you it gives you hope Even though the circumstances between the characters and the viewers are quite different you dont feel that far removed from what the characters are going through
Filtered:  ['i never seen amazing film since i saw the shawshank redemption shawshank encompasses friendships hardships hopes dreams and great movie moves gives hope even though circumstances characters viewers quite different dont feel far removed characters going']


Once again, we need to tokenize and encode the words. I use the tokenizer which was previously declared because we want to encode the words based on words that are known by the model.

In [21]:
tokenize_words = token.texts_to_sequences(filtered)
tokenize_words = pad_sequences(tokenize_words, maxlen=max_length, padding='post', truncating='post')
print(tokenize_words)

[[    1    40    38   400     4   143     1   120     2 12075  3294 12075
  17803  9262 10476  1914  1333    32    21     3  1038   313   350    11
     72  2211    30   719    88   185  5382   136   130  3732    30    80
      0     0     0     0     0     0     0     0     0     0     0     0
      0     0     0     0     0     0     0     0     0     0     0     0
      0     0     0     0     0     0     0     0     0     0     0     0
      0     0     0     0     0     0     0     0     0     0     0     0
      0     0     0     0     0     0     0     0     0     0     0     0
      0     0     0     0     0     0     0     0     0     0     0     0
      0     0     0     0     0     0     0     0     0     0     0     0
      0     0     0     0     0     0     0     0     0     0]]


This is the result of the prediction which shows the **confidence score** of the review statement.

In [22]:
result = loaded_model.predict(tokenize_words)
print(result)

[[0.9937773]]


If the confidence score is close to 0, then the statement is **negative**. On the other hand, if the confidence score is close to 1, then the statement is **positive**. I use a threshold of **0.7** to determine which confidence score is positive and negative, so if it is equal or greater than 0.7, it is **positive** and if it is less than 0.7, it is **negative**

In [23]:
if result >= 0.7:
    print('positive')
else:
    print('negative')

positive


In [24]:
import pickle
pickle.dump(token,open("tkn.sav", "wb"))